In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import yaml

sys.path.append("./")
from Finance.Data import Loader

# Load data
loader = Loader("data/")
data, info = loader.load("000300")
print(info)

# Load Network Config
an_config = yaml.load(open("an_config.yaml"))
print(an_config)
cn_config = yaml.load(open("cn_config.yaml"))
print(cn_config)

In [ ]:
# Build Actor-Critic Model
from Finance.PolicyNetwork import ActorNetwork, CriticNetwork

# Optimize GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Build Network
actor = ActorNetwork(sess, an_config)
critic = CriticNetwork(sess, cn_config)

actor.describe()
critic.describe()